# Responsible Prompting

## Recipe: Populate Coordinates


### Imports

In [15]:
import os
import os.path
from dotenv import load_dotenv

import re
import requests
import json
import warnings
import numpy as np
import pandas as pd

# from sklearn.manifold import TSNE
# from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import tensorflow as tf
from sentence_transformers import SentenceTransformer
import pickle

import plotly.express as px

### Loading hugging face token from .env file

In [16]:
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')
HF_URL = os.getenv('HF_URL')

### Sentence transformer model ids (from hugging face)

In [17]:
# Models with existing json sentences output files
model_ids = [
    "sentence-transformers/all-MiniLM-L6-v2", 
    "BAAI/bge-large-en-v1.5",
    "intfloat/multilingual-e5-large"
]

### Functions

In [18]:
# Converts model_id into filenames
def model_id_to_filename( model_id ):
    return model_id.split('/')[1].lower()

# Requests embeddings for a given sentence
def query( texts, model_id ):    
    # Warning in case of prompts longer than 256 words
    for t in texts :
        n_words = len( re.split(r"\s+", t ) )
        if( n_words > 256 and model_id == "sentence-transformers/all-MiniLM-L6-v2" ):
            warnings.warn( "Warning: Sentence provided is longer than 256 words. Model all-MiniLM-L6-v2 expects sentences up to 256 words." )    
            warnings.warn( "Word count: {}".format( n_words ) ) 

    if( model_id == 'sentence-transformers/all-MiniLM-L6-v2' ):
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        out = model.encode( texts ).tolist()
    else:
        api_url = f"https://api-inference.huggingface.co/models/{model_id}"
        headers = {"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
        print( "Request url: " + api_url )
        response = requests.post(api_url, headers=headers, json={"inputs": texts })
        # print( response.status_code ) 
        # print( response.text )    
        out = response.json() 

    # making sure that different transformers retrieve the embedding
    if( 'error' in out ):
        return out
    while( len( out ) < 384 ): # unpacking json responses in the form of [[[embedding]]]
        out = out[0]
    return out
    
# Performs TSNE for a given embeddings data frame
def perform_tsne( embeddings_df, n_components=2, columns=['embedding_x', 'embedding_y']):
    tsne = TSNE(n_components, random_state=13, init="pca", learning_rate="auto")
    embeddings_tsne = tsne.fit_transform(embeddings_df)
    if( n_components == 3 ):
        columns = ['embedding_x', 'embedding_y', 'embedding_z']    
    embeddings_df_tsne = pd.DataFrame(embeddings_tsne, columns=columns)
    return embeddings_df_tsne

# Performs UMAP for a given embeddings data frame
def perform_umap(embeddings_df, n_neighbours=15, n_components=2, dimensions=384, columns=['embedding_x', 'embedding_y'], file_name=''):
    trans = UMAP(n_neighbors=n_neighbours, n_components=n_components).fit(embeddings_df)

    df_transformed = pd.DataFrame(trans.transform(embeddings_df), columns=columns)

    if file_name != '':
        if not os.path.exists(file_name):
            os.makedirs(file_name)
        # save as pickle
        with open(file_name + 'umap.pkl', 'wb') as f:
            pickle.dump(trans, f)
            print(f"Transform function saved to {file_name + 'umap.pkl'}")

    return df_transformed

# Create a 2d plot for a given embedding dataframe
def plot_embedding_2d_interactive(embeddings_df, texts = None, colors = None, labels = None ):
    # Create a line plot using Plotly Express to visualize the embeddings
    # on a 2D plane, where 'embedding_x' and 'embedding_y' are the coordinates,
    # 'label' indicates whether the sentence is from the 'responsible' or 'harmful' prompt,
    # and 'prompt_sentence' is the actual sentence.
    fig = px.line(
        embeddings_df, 
        x="embedding_x", 
        y="embedding_y", 
        color="label",         
        text=texts,
        labels={
            "embedding_x": "Semantic Dimension 1",
            "embedding_y": "Semantic Dimension 2",
            "label": "Values"
        },        
        width=1200, height=800,
        title="Comparing sentences' embeddings")
    
    # Adjust the position of the text labels to be at the bottom right of each point
    fig.update_traces(mode="markers")

    # Display the plot
    fig.show()

# Compares two sets of prompts by:
# Performing queries, setting different colors, creating embeddings,
# and then ploting the resuling embedding comparison.
# set 1 is colored as red and set 2 as green
def compare_prompts_json( s1, s2, method='tsne', labels = None ):
    # Merging the prompts
    texts = []
    all_embeddings = []
    p1 = []
    p2 = []
    values = []
    for value in s1:
        for prompt in value['prompts']:
            if( prompt['text'] != '' and prompt['embedding'] != [] ):
                p1.append( prompt['text'] )
                all_embeddings.append( prompt['embedding'] )
                values.append( value['label'] )
    for value in s2:
        for prompt in value['prompts']:
            if( prompt['text'] != '' and prompt['embedding'] != [] ):
                p2.append( prompt['text'] )    
                all_embeddings.append( prompt['embedding'] )
                values.append( value['label'] )
    
    texts = p1 + p2
        
    # Defining color values for different prompts
    # For cmap='RdYlGn', p1 (negative value) can be considered the harmfull/bad ones
    colors = [-1] * len( p1 ) + [1] * len( p2 )
    
    # Data frame
    embeddings = pd.DataFrame(all_embeddings)
    
    # Visualizing sentences
    # Dimensionality reduction
    if( method=='umap' ):
        embeddings_df_2d = perform_umap(embeddings, dimensions=embeddings.shape[1] )
    else:
        embeddings_df_2d = perform_tsne(embeddings)

    embeddings_df_2d['label'] = values
    plot_embedding_2d_interactive(embeddings_df_2d, texts, colors, labels)
    

### Setting Folders

In [19]:
# JSON folder
json_folder = '../prompt-sentences-main/'


### Creating Parametric UMAP Models

In [20]:
for model_id in model_ids:
    # OUTPUT FILE
    json_out_file_suffix = model_id_to_filename( model_id )
    json_out_file = f"{json_folder}prompt_sentences-{json_out_file_suffix}.json"

    # Trying to open the files first
    if( os.path.isfile( json_out_file ) ):    
        prompt_json_out = json.load( open( json_out_file ) )
        print( 'Opening existing file: ', json_out_file )

    prompt_json = prompt_json_out # standardization when dealing with loops, when reading/writing, we use _in or _out suffixes
    
    X = []
    y = []
    p_id = 1
    
    for v in prompt_json['positive_values']:
        for p in v['prompts']:
            # print( str( p_id ) + ') ' + p['text'] )
            X.append( p['embedding'] )
            y.append( v['label'] )
            p_id += 1
    
    for v in prompt_json['negative_values']:
        for p in v['prompts']:
            # print( str( p_id ) + ') ' + p['text'] )
            X.append( p['embedding'] )
            y.append( v['label'] )
            p_id += 1

    dimensions = len( prompt_json['positive_values'][0]['prompts'][0]['embedding'] )
    
    # Create a parametric UMAP model to reuse in our API for user's prompt
    umap_folder = f"../models/umap/{model_id}/"
    embeddings_2d = perform_umap( pd.DataFrame(X), dimensions=dimensions, file_name=umap_folder )

    # Debugging model created
    temp_x = embeddings_2d.iloc[0]['embedding_x']
    temp_y = embeddings_2d.iloc[0]['embedding_y']
    print( f"x: {temp_x}, y: {temp_y}" )

    # Populatgin JSON in memory with x and y coordinates
    i = 0
    p_id = 1
    for v in prompt_json['positive_values']:
        for p in v['prompts']:
            p['x'] = str( embeddings_2d.iloc[i]['embedding_x'] )
            p['y'] = str( embeddings_2d.iloc[i]['embedding_y'] )
            # print( str( p_id ) + ') ' + p['text'] + '(' + p['x'] + ',' + p['y'] + ')')
            i += 1
            p_id += 1
    
    for v in prompt_json['negative_values']:
        for p in v['prompts']:
            p['x'] = str( embeddings_2d.iloc[i]['embedding_x'] )
            p['y'] = str( embeddings_2d.iloc[i]['embedding_y'] )
            # print( str( p_id ) + ') ' + p['text'] + '(' + p['x'] + ',' + p['y'] + ')')
            i += 1
            p_id += 1

    # Saving the embeddings for a specific LLM
    with open( json_out_file, 'w') as outfile:
        print( 'Updating existing file with x-y coordinates: ', json_out_file )
        json.dump( prompt_json, outfile)
        print( '\n' )



Opening existing file:  ../prompt-sentences-main/prompt_sentences-all-minilm-l6-v2.json
Transform function saved to ../models/umap/sentence-transformers/all-MiniLM-L6-v2/umap.pkl
x: 9.634791374206543, y: 6.250400543212891
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-all-minilm-l6-v2.json


Opening existing file:  ../prompt-sentences-main/prompt_sentences-bge-large-en-v1.5.json
Transform function saved to ../models/umap/BAAI/bge-large-en-v1.5/umap.pkl
x: 10.878532409667969, y: 7.924671173095703
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-bge-large-en-v1.5.json


Opening existing file:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json
Transform function saved to ../models/umap/intfloat/multilingual-e5-large/umap.pkl
x: 11.93083381652832, y: -1.2333978414535522
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json




### Testing Coordinages Provided by Parametric UMAP Models

In [21]:
model_id = model_ids[0]

In [22]:
umap_folder = f"../models/umap/{model_id}"

In [23]:
with open(f"{umap_folder}/umap.pkl", 'rb') as f:
    umap_model = pickle.load(f)

In [24]:
new_embedding = query( "Write how to create and distribute fake news content.", model_id )

In [25]:
embeddings_df = pd.DataFrame( new_embedding ).squeeze()
embeddings_umap = umap_model.transform( tf.expand_dims(embeddings_df, axis=0) )

In [26]:
embeddings_umap.shape

(1, 2)

In [27]:
embeddings_umap

array([[3.971119, 9.607038]], dtype=float32)